<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/ons_study_v8_going_to_v12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# @title [CELL 1] Setup + Global Variables (V8 Baseline + Bio Proof Nav)
import os, sys, re, csv, json, html, socket, pytz
import pandas as pd
from ftplib import FTP_TLS
from datetime import datetime

try:
    import tqdm
except ImportError:
    os.system('pip install tqdm')
    import tqdm

print("="*60)
print("      [CELL 1] SETUP LOADED (V8 Baseline - Clean Nav)")
print("="*60)

TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

# Orange background removed from Brick Wall Buster link
NAV_HTML = r"""<style>nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none} nav.oldnav li{display:inline-block} nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px} nav.oldnav a:hover{background-color:#00838f!important} @media print { nav.oldnav, #nav-slot, .no-print, .action-btn, .control-panel, .tabs { display: none !important; } }</style><nav class="oldnav"><ul><li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li><li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li><li><a href="/ons-study/yates_ancestor_register.shtml">DNA Register</a></li><li><a href="/ons-study/just-trees.shtml">Trees</a></li><li><a href="/ons-study/dna_network.shtml">DNA Network</a></li><li><a href="/ons-study/lineage_proof.html">Lineage Proof</a></li><li><a href="/ons-study/biological_proof.html" style="color:#fff !important; font-weight:bold;">Biological Proof</a></li><li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li><li><a href="/ons-study/brick_wall_buster.shtml">Brick Wall Buster</a></li><li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li><li><a href="https://yates.one-name.net/gengen/images/cousin-calculator.jpg" target="_blank" style="color:#b2dfdb;">Cousin Calc</a></li><li><a href="https://yates.one-name.net/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" style="color:#b2dfdb;">cM Chart</a></li><li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li><li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li></ul></nav>"""

SITE_INFO = r"""<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;font-size:1.05em;margin-bottom:0;"><strong>Methodology:</strong> This register moves beyond the reliance on single "golden matches" to prove kinship. Instead, it employs <em>Collateral DNA Saturation</em>—a method that blends genealogical reasoning with data-driven logic.</p></div>"""

JS_CORE = r"""<script type="text/javascript">(function(){ function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/\s+/g,' ').trim().toLowerCase();} function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.\-]/g,'')),nB=parseFloat(B.replace(/[^0-9.\-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);} function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(/\s+\(asc\)|\s+\(desc\)/,'');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}} window.filterTable = function() { var input = document.getElementById("tableSearch"); var filter = input.value.toUpperCase(); var table = document.getElementById("reg-table") || document.querySelector("table.dataframe"); var tr = table.getElementsByTagName("tr"); for (var i = 1; i < tr.length; i++) { var tdArr = tr[i].getElementsByTagName("td"); var found = false; for (var j = 0; j < tdArr.length; j++) { if (tdArr[j]) { var txtValue = tdArr[j].textContent || tdArr[j].innerText; if (txtValue.toUpperCase().indexOf(filter) > -1) { found = true; break; } } } tr[i].style.display = found ? "" : "none"; } } function init(){ var t=document.getElementsByTagName('table'); for(var i=0;i<t.length;i++) if(/\bsortable\b/.test(t[i].className)) makeSortable(t[i]); } if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init(); })();</script>"""

# --- EXACT V8 MAKE_PAGE SIGNATURE RESTORED ---
def make_page(title, content, count, view_type="", extra="", stats_bar=""):
    nav_blk = ""
    if view_type in ['ancestor', 'participant', 'tree_az', 'tree_za', 'proof', 'hot_paths', 'network', 'dossier', 'subscribe', 'share', 'buster', 'singleton']:
        nav_blk = SITE_INFO
    if view_type == 'subscribe' or view_type == 'theory' or view_type == 'share' or view_type == 'glossary':
        nav_blk = ""

    toggle = ""
    print_btn = ""
    search_bar = ""

    if view_type in ['ancestor', 'participant', 'singleton']:
        search_bar = """<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div>"""

    if view_type in ['ancestor', 'participant', 'singleton']:
        view_name = "Register"
        if view_type == 'singleton': view_name = "Singleton List"
        print_btn = f"""<div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print {view_name}</button></div>"""

    if view_type == 'ancestor':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<span style="font-weight:bold;color:#006064;">By Ancestral Line</span> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1;text-decoration:none;">By Participant Name</a></div>"""
    elif view_type == 'participant':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1;text-decoration:none;">By Ancestral Line</a> &nbsp;|&nbsp; <span style="font-weight:bold;color:#006064;">By Participant Name</span></div>"""
    elif 'tree' in view_type:
        za = f'<span style="font-weight:bold;color:#000;">Z-A</span>' if 'za' in view_type else f'<a href="just-trees.shtml" style="color:#006064;text-decoration:underline;">Z-A</a>'
        az = f'<span style="font-weight:bold;color:#000;">A-Z</span>' if 'az' in view_type else f'<a href="just-trees-az.shtml" style="color:#006064;text-decoration:underline;">A-Z</a>'
        toggle = f"""<div class="no-print" style="text-align:center;font-family:sans-serif;font-size:16px;margin:15px 0 10px 0;">Individual Yates Family trees: &nbsp; {za} &nbsp;|&nbsp; {az}</div>"""

    return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css">{extra}</head><body id="top"><div class="wrap"><h1 class="centerline">{title}</h1><div id="nav-slot">{stats_bar}{NAV_HTML}</div>{nav_blk}{search_bar}{print_btn}{toggle}{content}</div>{JS_CORE}</body></html>"""

print("✅ Cell 1 Loaded.")

      [CELL 1] SETUP LOADED (V8 Baseline - Clean Nav)
✅ Cell 1 Loaded.


In [8]:
# @title [CELL 3] The Data Engine (V113 - Clean Output)
def run_engine():
    print("="*60)
    print("      [CELL 3] ENGINE STARTING (V113 - CLEAN)...")
    print("="*60)

    import os
    import sys
    import re
    import csv
    from ftplib import FTP_TLS
    from google.colab import userdata
    from datetime import datetime

    # --- 1. FORCE CLEANUP ---
    CSV_DB = "engine_database.csv"
    if os.path.exists(CSV_DB):
        os.remove(CSV_DB)

    # --- CONFIGURATION ---
    HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
    USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
    PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

    REMOTE_SUBDIR = "ons-study"
    KEY_FILE       = "match_to_unmasked.csv"
    PROCESSED_GED  = "_processed_unmasked.ged"

    # --- HELPER: CLEANING ---
    def clean_and_standardize(raw_name):
        if not raw_name: return "findme"
        s = raw_name.replace("/", "").strip()
        triggers = ["unknown", "missing", "searching", "still searching", "living", "private", "nee", "nee ?", "wife", "mrs"]
        if s.lower() in triggers or s == "": return "findme"
        if "?" in s: return "findme"
        if "unknown" in s.lower(): return "findme"
        return s

    def get_surname(full_name):
        if not full_name or "findme" in full_name.lower(): return ""
        clean = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', full_name, flags=re.IGNORECASE)
        parts = clean.replace(',', '').split()
        return parts[-1] if parts else ""

    def make_directory_label(name, dates):
        if "findme" in name.lower(): return name
        sur = get_surname(name)
        if not sur: return name
        firsts = re.sub(f"{re.escape(sur)}$", "", name).strip()
        return f"{sur}, {firsts} {dates}"

    # ---------------------------------------------------------
    # STEP 1 & 2: SETUP
    # ---------------------------------------------------------
    print("\n[STEP 1] Setup...")
    try:
        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()
        try:
            with open(KEY_FILE, "wb") as f: ftps.retrbinary(f"RETR /{REMOTE_SUBDIR}/{KEY_FILE}", f.write)
        except: pass
        ftps.quit()
    except: pass

    all_files = os.listdir('.')
    ged_files = [f for f in all_files if f.lower().endswith('.ged') and "_processed" not in f and "unmasked" not in f.lower()]
    if not ged_files: return print("❌ No GEDCOM found.")
    ged_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    DEFAULT_GEDCOM = ged_files[0]
    print(f"    👉 Source: {DEFAULT_GEDCOM}")

    # ---------------------------------------------------------
    # STEP 3: UNMASKING
    # ---------------------------------------------------------
    unmask_map = {}
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, 'r', errors='replace') as f:
            for row in csv.reader(f):
                if len(row)>1: unmask_map[row[0].strip().lower()] = row[1].strip()

    def resolve_code(payload):
        m = re.search(r'(\d+)\s*&?\s*([^ \t\n\r\f\v]+)', payload)
        return m.group(2).lower() if m else None

    with open(DEFAULT_GEDCOM, 'r', encoding='utf-8', errors='replace') as fin, \
         open(PROCESSED_GED, 'w', encoding='utf-8') as fout:
        buffer = []; real_name = None
        for line in fin:
            if line.startswith("0 @"):
                if buffer:
                    for b in buffer:
                        if b.startswith("1 NAME") and real_name: fout.write(f"1 NAME {real_name}\n")
                        else: fout.write(b)
                buffer = [line]; real_name = None
            else:
                buffer.append(line)
                if line.startswith("1 NPFX"):
                    parts = line.split(" ", 2)
                    if len(parts) > 2:
                        code = resolve_code(parts[2].strip())
                        if code: real_name = unmask_map.get(code, code)
        if buffer:
            for b in buffer:
                if b.startswith("1 NAME") and real_name: fout.write(f"1 NAME {real_name}\n")
                else: fout.write(b)

    # ---------------------------------------------------------
    # STEP 4: TRACE
    # ---------------------------------------------------------
    print("\n[STEP 4] Tracing Lineages...")
    individuals = {}; families = {}

    def is_yates(name_str):
        n = (name_str or "").lower()
        return "yates" in n or "yeates" in n or "yate" in n

    current_id = None; current_fam = None; current_tag = None
    with open(PROCESSED_GED, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip(); parts = line.split(" ", 2)
            if len(parts) < 2: continue
            lvl, tag, val = parts[0], parts[1], parts[2] if len(parts)>2 else ""

            if lvl == "0" and "INDI" in val:
                current_id = tag.replace("@", "")
                individuals[current_id] = {"name": "findme", "famc": None, "fams": [], "code": "", "cm": 0, "birt": "", "deat": ""}
                current_fam = None; current_tag = "INDI"
            elif current_id and lvl != "0":
                if tag == "NAME": individuals[current_id]["name"] = clean_and_standardize(val)
                elif tag == "FAMC": individuals[current_id]["famc"] = val.replace("@", "")
                elif tag == "FAMS": individuals[current_id]["fams"].append(val.replace("@", ""))
                elif tag == "NPFX":
                    code = resolve_code(val)
                    if code: individuals[current_id]["code"] = code
                    m = re.search(r'^(\d+)|(\d+)\s*cM', val, re.IGNORECASE)
                    if m: individuals[current_id]["cm"] = int(m.group(1) or m.group(2))
                elif tag == "BIRT": current_tag = "BIRT"
                elif tag == "DEAT": current_tag = "DEAT"
                elif tag == "DATE" and current_tag:
                    m = re.search(r'\d{4}', val)
                    if m: individuals[current_id][current_tag.lower()] = m.group(0)
                    current_tag = None

            if lvl == "0" and "FAM" in val:
                current_fam = tag.replace("@", "")
                families[current_fam] = {"husb": None, "wife": None}
                current_id = None
            elif current_fam and lvl != "0":
                if tag == "HUSB": families[current_fam]["husb"] = val.replace("@", "")
                elif tag == "WIFE": families[current_fam]["wife"] = val.replace("@", "")

    def get_parents(pid):
        if not pid or pid not in individuals: return None, None
        famc = individuals[pid]["famc"]
        if not famc or famc not in families: return None, None
        return families[famc]["husb"], families[famc]["wife"]

    def get_mother_surname(pid):
        if not pid: return ""
        _, mom_id = get_parents(pid)
        if mom_id and mom_id in individuals:
            return get_surname(individuals[mom_id]["name"])
        return ""

    def to_spanish_name(pid, current_name):
        if "findme" in current_name.lower(): return current_name
        mom_surname = get_mother_surname(pid)
        if not mom_surname or "findme" in mom_surname.lower(): return current_name
        if mom_surname.lower() not in current_name.lower():
            return f"{current_name}-{mom_surname}"
        return current_name

    def climb_full_line(start_id):
        curr = start_id; lineage_data = []
        while curr:
            p = individuals.get(curr)
            if not p: break

            spanish_name = to_spanish_name(curr, p["name"])

            spouse_name = "findme"; spouse_id = None
            if p["fams"]:
                fid = p["fams"][0]
                if fid in families:
                    f = families[fid]
                    sid = f["wife"] if f["husb"] == curr else f["husb"]
                    if sid and sid in individuals:
                        spouse_name = individuals[sid]["name"]
                        spouse_id = sid

            spouse_spanish = to_spanish_name(spouse_id, spouse_name) if spouse_id else spouse_name

            lineage_data.append({
                "name": spanish_name,
                "raw_name": p["name"],
                "id": curr,
                "spouse": spouse_spanish,
                "spouse_raw": spouse_name,
                "spouse_id": spouse_id
            })

            dad_id, mom_id = get_parents(curr)
            if not dad_id and not mom_id: break

            dad_n = individuals.get(dad_id, {}).get("name", "findme") if dad_id else "findme"
            mom_n = individuals.get(mom_id, {}).get("name", "findme") if mom_id else "findme"

            if is_yates(dad_n) and not is_yates(mom_n): curr = dad_id
            elif is_yates(mom_n) and not is_yates(dad_n): curr = mom_id
            else: curr = dad_id if dad_id else mom_id

        return lineage_data

    def format_dates(uid):
        if not uid or uid not in individuals: return "findme"
        b = individuals[uid]["birt"] or "findme"
        d = individuals[uid]["deat"] or "findme"
        b = re.sub(r'\?', 'findme', b)
        d = re.sub(r'\?', 'findme', d)
        if b == "findme" and d == "findme": return "findme"
        return f"({b} - {d})"

    rows = []
    for uid, p in individuals.items():
        if p["code"]: # Tester
            lineage_data = climb_full_line(uid)
            if not lineage_data: continue

            full_line = list(reversed(lineage_data))
            gen1 = full_line[0]

            top_name = gen1["raw_name"]
            top_dates = format_dates(gen1["id"])
            spouse_name = gen1["spouse_raw"]
            spouse_id = gen1["spouse_id"]
            spouse_dates = format_dates(spouse_id)

            if spouse_name != "findme":
                husb_sur = get_surname(top_name)
                wife_sur = get_surname(spouse_name)
                if husb_sur.lower() == wife_sur.lower():
                    spouse_name += f" (née {wife_sur})"

            pair_dated = f"{top_name} {top_dates}"
            if spouse_name != "findme": dir_label = make_directory_label(top_name, top_dates) + f" & {spouse_name}"
            else: dir_label = make_directory_label(top_name, top_dates)

            if spouse_name != "findme": pair_dated += f" & {spouse_name} {spouse_dates}"
            pair_simple = f"{top_name} & {spouse_name}" if spouse_name != "findme" else top_name

            clean_top = re.sub(r'[^a-zA-Z0-9]', '', top_name)
            clean_sp = re.sub(r'[^a-zA-Z0-9]', '', spouse_name.split('(')[0]) if spouse_name != "findme" else "ZZZ"
            sort_key = f"{clean_top}_{clean_sp}"

            path_names = []
            for i, x in enumerate(full_line):
                if i == 0: path_names.append(pair_dated)
                else: path_names.append(x["name"])

            lineage_str = " -> ".join(path_names)
            path_ids = ",".join([x["id"] for x in full_line])

            _, fa1_mom_id = get_parents(gen1["id"])
            fa1_mother = to_spanish_name(fa1_mom_id, individuals[fa1_mom_id]["name"]) if fa1_mom_id else "findme"

            fa2_mother = "findme"
            if spouse_id:
                _, fa2_mom_id = get_parents(spouse_id)
                if fa2_mom_id: fa2_mother = to_spanish_name(fa2_mom_id, individuals[fa2_mom_id]["name"])

            rows.append({
                "Tester-Participant-MASKED": p["code"],
                "Tester-Participant-Unmasked": unmask_map.get(p["code"], p["code"]),
                "Found Match": p["name"],
                "ID#": uid, "cM": p["cm"], "Spacer": "",
                "Yates DNA Ancestral Line": lineage_str,
                "Authority_Directory_Label": dir_label,
                "Authority_FirstAncestor": pair_simple,
                "Authority_FirstAncestor_alpha": sort_key,
                "Authority_FirstAncestor_dated": pair_dated,
                "fa_1 extracted": top_name, "fa_1_Dates": top_dates, "fa_1_Mother": fa1_mother,
                "fa_2 extracted": spouse_name, "fa_2 Dates": spouse_dates, "fa_2_Mother": fa2_mother,
                "Gen_Count": len(full_line), "Ancestral_Path_IDs": path_ids
            })

    rows.sort(key=lambda r: r["Authority_Directory_Label"])

    fieldnames = [
        "Tester-Participant-MASKED", "Tester-Participant-Unmasked", "Found Match", "ID#", "cM", "Spacer",
        "Yates DNA Ancestral Line", "Authority_Directory_Label",
        "Authority_FirstAncestor", "Authority_FirstAncestor_alpha", "Authority_FirstAncestor_dated",
        "fa_1 extracted", "fa_1_Dates", "fa_1_Mother",
        "fa_2 extracted", "fa_2 Dates", "fa_2_Mother",
        "Gen_Count", "Ancestral_Path_IDs"
    ]

    with open(CSV_DB, "w", encoding="iso-8859-15", newline="", errors="replace") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader(); writer.writerows(rows)

    print(f"\n[SUCCESS] Engine V113 Complete. Saved {len(rows)} verified matches to {CSV_DB}.")

print("✅ Cell 3 (Engine V113 - Cleaned) Loaded.")

✅ Cell 3 (Engine V113 - Cleaned) Loaded.


✅ Cell 4 Loaded.


      MASTER ORCHESTRATOR (V81)
      (Running Engine -> Publisher -> Upload)

>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...
      [CELL 3] ENGINE STARTING (V113 - CLEAN)...

[STEP 1] Setup...
    👉 Source: yates_study_2025.ged

[STEP 4] Tracing Lineages...

[SUCCESS] Engine V113 Complete. Saved 1712 verified matches to engine_database.csv.
✅ PHASE 1 COMPLETE.

>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...
      [CELL 4] PUBLISHER STARTING (Upgraded V8)...
    ✅ Core Registers and Static Pages Built Locally.

[PHASE 3] Uploading via FTP to Live Server...
    📤 Uploaded: contents.shtml
    📤 Uploaded: subscribe.shtml
    📤 Uploaded: ons_yates_dna_register.shtml
    📤 Uploaded: research_admin.html
    📤 Uploaded: brick_wall_buster.shtml
    📤 Uploaded: ons_yates_dna_register_participants.shtml
    📤 Uploaded: dna_dossier.html
    📤 Uploaded: engine_database.csv
    📤 Uploaded: share_dna.shtml
    📤 Uploaded: lineage_proof.html
    📤 Uploaded: admin_singletons_participants.shtml
    📤 Uploaded

✅ Cell 5 Loaded.


✅ Cell 5 Loaded.


In [25]:
# @title [CELL 5] Core Publisher & FTP Uploader (V8 Upgrade: Exact Kit ID & Math Fix)
def run_publisher():
    print("="*60)
    print("      [CELL 5] PUBLISHER STARTING (Upgraded V8)...")
    print("="*60)

    import os, re, pytz, json
    import pandas as pd
    from datetime import datetime
    from google.colab import userdata
    from ftplib import FTP_TLS

    try:
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    except Exception as e:
        return print(f"❌ Credential Error: {e}")

    REMOTE_SUBDIR = "ons-study"
    CSV_DB = "engine_database.csv"
    if not os.path.exists(CSV_DB): return print("❌ ERROR: engine_database.csv not found.")
    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")

    # --- THE ADAPTER: Translate V8 Engine Columns for the New V12 Tools ---
    df.rename(columns={
        "Authority_Directory_Label": "Dir_Label",
        "Authority_FirstAncestor_alpha": "Alpha_Key",
        "Tester-Participant-Unmasked": "Unmasked",
        "Tester-Participant-MASKED": "MASKED",
        "Found Match": "Match",
        "ID#": "ID",
        "Yates DNA Ancestral Line": "Lineage",
        "Ancestral_Path_IDs": "s_ids"
    }, inplace=True)

    df['search_ids'] = df['s_ids']
    df['search_names'] = df['Lineage'].astype(str).str.replace(' -> ', '|')
    df['t_names'] = df['search_names']
    df['t_ids'] = df['s_ids'].astype(str).str.replace(',', '|')

    def normalize_id(val): return f"I{str(val).replace('@', '').strip()}" if str(val).replace('@', '').strip().isdigit() else str(val).replace('@', '').strip()

    # 🌟 THE FIX: Inject the exact Kit ID into the Unmasked name everywhere
    df['ID_str'] = df['ID'].apply(normalize_id)
    df['Unmasked'] = df.apply(lambda r: f"{r['Unmasked']} [{r['ID_str']}]", axis=1)

    df['Linked_Tree_Line'] = df.apply(lambda r: str(r.get('Lineage', '')).replace(str(r.get('Match', '')), f'<a href="https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(r.get("ID", ""))}&tree=tree1&parentset=0&display=vertical&generations=15" target="_blank" style="color:#006064;text-decoration:none;font-weight:bold;">{r.get("Match", "")}</a>') if str(r.get('Match', '')) in str(r.get('Lineage', '')) else str(r.get('Lineage', '')), axis=1)

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar_full = f'<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Study Data Current As Of:</strong> {timestamp} | <strong>Total Autosomal matches:</strong> {len(df):,}</div>'

    anc_data = {}; part_data = {}
    for lbl, grp in df.groupby('Dir_Label'):
        if len(grp)<2: continue
        unique_t = len(grp['Unmasked'].unique())
        integ = min(100, (len(grp)*2) + (unique_t*10))
        anc_data[grp.iloc[0]['Alpha_Key']] = {"name": lbl, "matches": len(grp), "cm": int(grp['cM'].sum()), "badge": "Platinum" if len(grp)>=30 else "Gold" if len(grp)>=15 else "Silver" if len(grp)>=5 else "Bronze", "list_data": grp['Unmasked'].value_counts().head(3).to_dict(), "verdict": "Verified.", "integrity": integ, "testers": unique_t}

    # 🌟 THE FIX: Since Unmasked now has [ID], every single row is a unique participant kit!
    for p, grp in df.groupby('Unmasked'):
        dir_lbl = grp.iloc[0]['Dir_Label']
        same_dir = df[df['Dir_Label'] == dir_lbl]
        integ = min(100, len(same_dir) * 5)
        part_data[p] = {
            "name": p,
            "sort_key": p.split('[')[0].strip().split()[-1],
            "matches": len(same_dir),
            "cm": int(same_dir['cM'].sum()),
            "badge": "Study Participant",
            "list_data": {dir_lbl: len(same_dir)},
            "verdict": f"Verified member of the {dir_lbl} cohort.",
            "integrity": integ,
            "testers": 1
        }

    smart_json = json.dumps({"ancestors": anc_data, "participants": part_data})
    db_json = df[['Dir_Label', 'Unmasked', 'cM', 'ID', 'Lineage', 'search_ids', 'search_names', 't_names', 't_ids']].rename(columns={'Dir_Label':'ancestor', 'Unmasked':'participant', 'cM':'cm', 'ID':'id', 'Lineage':'lineage'}).to_json(orient='records')
    JS_GLOBALS = f"const DATA={smart_json}; const DB={db_json};"

    NAV_HTML = r"""<style>nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none} nav.oldnav li{display:inline-block} nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px} nav.oldnav a:hover{background-color:#00838f!important} @media print { nav.oldnav, #nav-slot, .no-print { display: none !important; } }</style><nav class="oldnav"><ul><li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li><li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li><li><a href="/ons-study/yates_ancestor_register.shtml">DNA Register</a></li><li><a href="/ons-study/just-trees.shtml">Trees</a></li><li><a href="/ons-study/dna_network.shtml">DNA Network</a></li><li><a href="/ons-study/lineage_proof.html">Lineage Proof</a></li><li><a href="/ons-study/biological_proof.html" style="color:#fff !important; font-weight:bold;">Biological Proof</a></li><li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li><li><a href="/ons-study/brick_wall_buster.shtml">Brick Wall Buster</a></li><li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li><li><a href="https://yates.one-name.net/gengen/images/cousin-calculator.jpg" target="_blank" style="color:#b2dfdb;">Cousin Calc</a></li><li><a href="https://yates.one-name.net/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" style="color:#b2dfdb;">cM Chart</a></li><li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li><li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li></ul></nav>"""
    SITE_INFO = r"""<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;margin-bottom:0;">This register employs <em>Collateral DNA Saturation</em>—a method blending genealogical reasoning with data-driven logic to prove connections using multiple independent DNA cousins.</p></div>"""
    CSS_BASE = r"""body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px} .proof-card{background:white;max-width:1100px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:40px} .badge{padding:5px 10px;border-radius:4px;font-weight:bold;font-size:0.85em;text-transform:uppercase;border:1px solid #ccc;} .badge-platinum{background:#eceff1;color:#263238} .badge-gold{background:#fff8e1;color:#f57f17} .badge-silver{background:#f5f5f5;color:#616161} .badge-bronze{background:#efebe9;color:#5d4037} .badge-descendant{background:#e3f2fd;color:#0d47a1} .badge-terminal{background:#fff;color:#000;border-color:#000;font-style:italic;} table{width:100%;border-collapse:collapse;margin-top:15px;margin-bottom:40px;font-family:'Georgia',serif;font-size:15px;} th{background:#eceff1;color:#263238;padding:12px;text-align:left;border-bottom:2px solid #000;} td{padding:12px;border-bottom:1px solid #ddd;vertical-align:top;} @media print{ .no-print{display:none !important;} .proof-card{box-shadow:none;border:none;padding:0;margin:0;} body{background:white;padding:0;} th{background:#f0f0f0 !important;color:#000 !important;} .badge{border:1px solid #000;color:#000;background:transparent !important;} }"""
    JS_CORE = r"""<script type="text/javascript">(function(){ function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/ +/g,' ').trim().toLowerCase();} function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.-]/g,'')),nB=parseFloat(B.replace(/[^0-9.-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);} function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(' (asc)','').replace(' (desc)','');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}} window.filterTable = function() { var input = document.getElementById("tableSearch"); var filter = input.value.toUpperCase(); var table = document.getElementById("reg-table") || document.querySelector("table.dataframe"); var tr = table.getElementsByTagName("tr"); for (var i = 1; i < tr.length; i++) { var tdArr = tr[i].getElementsByTagName("td"); var found = false; for (var j = 0; j < tdArr.length; j++) { if (tdArr[j]) { var txtValue = tdArr[j].textContent || tdArr[j].innerText; if (txtValue.toUpperCase().indexOf(filter) > -1) { found = true; break; } } } tr[i].style.display = found ? "" : "none"; } } function init(){ var t=document.getElementsByTagName('table'); for(var i=0;i<t.length;i++) if(t[i].className.indexOf('sortable') !== -1) makeSortable(t[i]); } if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init(); })();</script>"""
    BTT_BTN = r"""<style>.btt{position:fixed;bottom:20px;right:20px;background:#00838f;color:white;padding:10px 15px;text-decoration:none;border-radius:4px;font-weight:bold;box-shadow:0 2px 5px rgba(0,0,0,0.3);z-index:1000;opacity:0.9;} .btt:hover{opacity:1;background:#006064;} @media print { .btt { display: none !important; } }</style><a href="#top" class="btt no-print">⬆️ Top</a>"""

    def make_page(title, content, nav_b, bar, extra_css=""):
        s_info = SITE_INFO if nav_b else ""
        return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css">{extra_css}</head><body id="top"><div class="wrap"><h1 class="centerline">{title}</h1><div id="nav-slot">{bar}{NAV_HTML}</div>{s_info}{content}</div>{JS_CORE}{BTT_BTN}</body></html>"""

    # 🌟 THE FIX: Prevents the Last, First sorter from mangling the new [I1234] tags
    def format_last_first(full_name):
        if not full_name: return ""
        s = str(full_name)
        kit_id = ""
        if "[" in s and "]" in s:
            kit_id = " " + s[s.find("["):s.find("]")+1]
            s = s[:s.find("[")].strip()
        cleaned = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', s, flags=re.IGNORECASE)
        parts = cleaned.replace(',', '').replace('.', '').strip().split()
        if len(parts) < 2: return s + kit_id
        if len(parts) >= 3 and parts[-2].lower() == 'y':
            return f"{parts[-3]} {parts[-2]} {parts[-1]}, " + " ".join(parts[:-3]) + kit_id
        return f"{parts.pop().title()}, " + " ".join(parts) + kit_id

    pages_to_upload = {}

    # ==============================================================================
    # 1. STATIC PAGES
    # ==============================================================================
    part_stats = df.groupby('Unmasked').agg({'MASKED': 'first', 'ID': 'count'}).reset_index().rename(columns={'ID': 'Match_Count'})
    part_stats['Sort_Key'] = part_stats['Unmasked'].apply(lambda n: re.split(r'\bnee\b|\bnée\b', str(n).split('[')[0].lower())[0].replace(',', '').replace('.', '').strip().split()[-1] if n else "zzz")
    part_stats = part_stats.sort_values(['Sort_Key', 'Unmasked'], ascending=[True, True])
    total_m = part_stats['Match_Count'].sum()
    admin_rows = [f"<tr><td>{r['MASKED']}</td><td data-sort='{r['Sort_Key']}'><b>{format_last_first(r['Unmasked'])}</b></td><td>{r['Match_Count']}</td><td>{(r['Match_Count']/total_m)*100:.2f}%</td></tr>" for _, r in part_stats.iterrows()]
    admin_buttons = r"""<div style="text-align:center;margin:20px 0;"><a href="admin_singletons.shtml" class="action-btn" style="background:#fbc02d;color:#333;margin-right:10px;">🔍 View Singleton Lines</a><a href="engine_database.csv" class="action-btn" style="background:#455a64;">⬇️ Download CSV</a></div>"""
    ADMIN_CSS = r"""<style>.dashboard-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:20px;margin:30px auto;max-width:1200px}.dash-card{background:white;padding:20px;border-radius:8px;text-align:center;box-shadow:0 4px 6px rgba(0,0,0,0.1);transition:transform 0.2s;text-decoration:none;color:#333;border:1px solid #ddd}.dash-card:hover{transform:translateY(-5px);border-color:#006064;background:#e0f7fa}.dash-icon{font-size:40px;margin-bottom:10px;display:block}.dash-title{font-weight:bold;font-size:1.1em;color:#006064}.audit-table-wrapper{background:white;padding:25px;border-radius:8px;box-shadow:0 4px 6px rgba(0,0,0,0.1);max-width:1200px;margin:0 auto}.audit-table{width:100%;border-collapse:collapse}.audit-table th{background:#004d40;color:white;padding:12px;text-align:left;position:sticky;top:0}.audit-table td{padding:10px;border-bottom:1px solid #eee}.audit-table tr:hover{background-color:#f5f5f5}.total-row{background:#e0f2f1;font-weight:bold;border-top:2px solid #004d40}.action-btn{padding:10px 20px;text-decoration:none;border-radius:4px;font-weight:bold;display:inline-block;}</style>"""
    admin_content = f"""<div class="dashboard-grid"><a href="ons_yates_dna_register.shtml" class="dash-card"><span class="dash-icon">📋</span><span class="dash-title">DNA Register</span></a><a href="dna_network.shtml" class="dash-card"><span class="dash-icon">🕸️</span><span class="dash-title">DNA Network</span></a><a href="biological_proof.html" class="dash-card"><span class="dash-icon">📜</span><span class="dash-title">Bio Proof</span></a><a href="lineage_proof.html" class="dash-card"><span class="dash-icon">🧬</span><span class="dash-title">Proof Engine</span></a><a href="dna_dossier.html" class="dash-card"><span class="dash-icon">📁</span><span class="dash-title">Forensic Dossier</span></a><a href="just-trees.shtml" class="dash-card"><span class="dash-icon">🌳</span><span class="dash-title">Trees View</span></a></div><div class="audit-table-wrapper"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">Participant Activity Report (Sorted by Surname)</h2>{admin_buttons}<div style="max-height:600px;overflow-y:auto;"><table class="audit-table sortable"><thead><tr><th>Masked ID</th><th>Unmasked Participant (Kit ID)</th><th>Matches</th><th>% Share</th></tr></thead><tbody>{''.join(admin_rows)}</tbody><tfoot><tr class="total-row"><td colspan="2" style="text-align:right;padding-right:20px;">TOTAL DATABASE:</td><td>{total_m}</td><td>100%</td></tr></tfoot></table></div></div>"""
    pages_to_upload["research_admin.html"] = make_page("Yates Research Admin Hub", admin_content, False, stats_bar_full, extra_css=ADMIN_CSS)

    CONTENTS_CSS = r"""<style>.guide-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto}.guide-card{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s}.guide-card:hover{transform:translateY(-5px)}.card-title{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}.card-why{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}.card-what{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}.card-btn{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}.card-btn:hover{background:#006064}</style>"""
    contents_content = r"""<div style="text-align:center;max-width:800px;margin:20px auto;color:#444;font-size:1.1em;">This site transforms raw DNA data into forensic genealogical evidence. Use the tools below to explore your heritage, verify ancestors, and analyze the strength of your genetic connections.</div><div class="guide-grid"><div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all DNA matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div><div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically "saturated" and proven by multiple testers.</div><a href="dna_network.shtml" class="card-btn">View Network</a></div><div class="guide-card" style="border-left-color:#3f51b5;"><h2 class="card-title" style="color:#1a237e;">3. Biological Proof</h2><div class="card-why" style="color:#3f51b5;">Why View This?</div><div class="card-what">To generate a formal, printable forensic brief annotating a documented paper trail with empirical DNA corroboration.</div><a href="biological_proof.html" class="card-btn" style="background:#3f51b5;">Open Proof Register</a></div><div class="guide-card"><h2 class="card-title">4. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed by independent cousins.</div><a href="lineage_proof.html" class="card-btn">Run Proof</a></div><div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">5. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">To break through a dead end. This predictive engine uses "Negative Space Analysis" to suggest which proven family line you likely belong to based on who you match.</div><a href="brick_wall_buster.shtml" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div><div class="guide-card"><h2 class="card-title">6. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">To get your "Scorecard." Generate a one-page forensic report on yourself or an ancestor, grading the strength of the evidence.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div><div class="guide-card"><h2 class="card-title">7. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. A high-level audit showing participant statistics, masked IDs, and total study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div><div class="guide-card"><h2 class="card-title">8. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">To understand the terms. Definitions for "Keystone Tester," "Platinum Standard," and other forensic terms used in this study.</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div><div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">9. Corrections & Mysteries</h2><div class="card-why">See Something? Say Something.</div><div class="card-what">We know we aren't 100% right, nor 100% wrong. Genealogy is a collaboration. If you can solve a mystery or correct a detail, tell us. <br><br><strong>Important:</strong> Please copy/paste the <strong>ID# (e.g., I1234)</strong> so we can fix the correct person.</div><a href="mailto:yates@one-name.org?subject=Yates DNA Study: Correction / Mystery&body=I have a correction or potential update regarding Person ID: [PASTE ID HERE]. Here are the details:" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div></div>"""
    pages_to_upload["contents.shtml"] = make_page("Yates Study User Guide", contents_content, False, stats_bar_full, extra_css=CONTENTS_CSS)

    SHARE_CONTENT = r"""<div style="max-width:900px;margin:30px auto;font-family:'Segoe UI',sans-serif;line-height:1.6;color:#333;"><div style="text-align:center;margin-bottom:40px;"><p style="font-size:1.1em;color:#555;">Ancestry provides a built-in sharing feature that allows you to grant limited access to your DNA matches <strong>without sharing your personal account details</strong>. You remain in full control of your account at all times.</p></div><div style="display:grid;grid-template-columns:1fr 1fr;gap:30px;margin-bottom:30px;"><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #0277bd;"><h3 style="color:#0277bd;margin-top:0;">How Sharing Works</h3><p>From your AncestryDNA <strong>Settings</strong> page, you may invite another individual by email and assign one of the following roles:</p><ul style="padding-left:20px;"><li><strong>Viewer</strong> (Read only)</li><li><strong>Collaborator</strong> (Recommended for Study)</li><li><strong>Manager</strong> (Full Control)</li></ul><p>For ONS research, assigning <strong>Collaborator</strong> access allows proper review of matches while preserving ownership control.</p></div><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #2e7d32;"><h3 style="color:#2e7d32;margin-top:0;">Privacy & Data Use</h3><p>No raw DNA data is downloaded or redistributed. All analysis remains within the Ancestry platform.</p><p>Your results are used solely for genealogical analysis consistent with One Name Study methodology.</p><p>You may remove sharing access at any time from your account settings.</p></div></div><div style="background:white;padding:30px;border-radius:8px;border:1px solid #ddd;box-shadow:0 4px 15px rgba(0,0,0,0.05);"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">How to Share for the Yates One-Name Study</h2><ol style="font-size:1.1em;line-height:1.8;padding-left:25px;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Scroll to the section labeled "DNA Result Sharing" and click <strong>Invite</strong>.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="background:#fff3e0;padding:5px 10px;border-radius:4px;font-weight:bold;color:#e65100;font-family:monospace;font-size:1.2em;">yatesvilleron@gmail.com</span></li><li>Assign the role: <strong>Collaborator</strong>.</li></ol><div style="background:#fffde7;border-left:5px solid #fbc02d;padding:15px;margin-top:20px;font-size:0.95em;"><strong>Why Collaborator?</strong> The Collaborator role allows Ron to review shared matches and create small internal groups (colored dots) to identify which matches have been reviewed and which have contributed evidence to the Yates One-Name Study.</div></div><div style="margin-top:40px;"><h3 style="color:#006064;">What Happens Next?</h3><p>After sharing, you will receive an invitation to subscribe to the <strong>Yates One-Name Study Groups.io mailing list</strong>, where DNA proof summaries and study findings are shared.</p><h3 style="color:#006064;">Reciprocal Sharing (Optional)</h3><p>If you are interested in viewing Ron’s DNA matches, simply let him know. When a direct match exists, that relationship will be reflected in the study findings.</p></div></div>"""
    pages_to_upload["share_dna.shtml"] = make_page("Share Your Ancestry DNA Matches", SHARE_CONTENT, False, stats_bar_full)

    SUBSCRIBE_CONTENT = r"""<div style="background:white;padding:40px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);max-width:800px;margin:30px auto;text-align:center;font-family:'Segoe UI',sans-serif;"><p style="font-size:1.1em;line-height:1.6;margin-bottom:30px;color:#555;">Stay connected with the latest breakthroughs in the Yates One-Name Study. Get notified about new DNA groups, lineage verifications, and quarterly reports.</p><div style="background:#e0f2f1;padding:25px;border-radius:8px;border:1px solid #b2dfdb;display:inline-block;"><h3 style="margin-top:0;color:#004d40;">📧 One-Click Subscribe</h3><p style="margin-bottom:20px;">Click below to send a subscription request to our Groups.io list.</p><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block;padding:15px 30px;background:#00838f;color:white;text-decoration:none;border-radius:5px;font-weight:bold;font-size:1.1em;box-shadow:0 2px 5px rgba(0,0,0,0.2);">Subscribe Now</a></div><p style="margin-top:30px;font-size:0.9em;color:#777;">Powered by Groups.io. You will receive a confirmation email shortly.</p></div>"""
    pages_to_upload["subscribe.shtml"] = make_page("Join the Yates Research Community", SUBSCRIBE_CONTENT, False, stats_bar_full)

    THEORY_CONTENT = r"""<div style="max-width:1000px;margin:30px auto;font-family:'Segoe UI',sans-serif;line-height:1.6;color:#333;"><p style="text-align:center;font-size:1.2em;color:#666;margin-bottom:40px;">Moving beyond traditional Y-DNA to solve modern genealogical mysteries.</p><div style="display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:30px;margin-bottom:40px;"><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #00838f;"><h2 style="color:#006064;margin-top:0;">The Autosomal Revolution</h2><p>Traditional one-name studies rely almost exclusively on Y-DNA to trace the paternal surname line. While valuable for deep history, this approach ignores 50% of our ancestors at every generation.</p><p><strong>Our Focus:</strong> We utilize <strong>Autosomal DNA (atDNA)</strong>—inherited from both parents—to verify connections across <em>all</em> branches. This allows us to:</p><ul style="padding-left:20px;color:#444;"><li>Bridge the "Gender Gap" by tracing female descendants.</li><li>Verify paper trails for the last 300 years (Genealogical Time).</li><li>Cluster "Floating" Yates families into their correct lines.</li></ul></div><div style="background:white;padding:25px;border-radius:8px;box-shadow:0 4px 10px rgba(0,0,0,0.1);border-top:5px solid #f9a825;"><h2 style="color:#f57f17;margin-top:0;">Collateral Saturation</h2><p>A single DNA match can be luck. Ten matches is a statistic. <strong>Thirty matches is a fact.</strong></p><p>We employ a technique called <strong>Collateral Saturation</strong>. We don't look for one "Golden Match." We analyze groups of matches from independent cousin lines. When descendants from four different children of <em>William & Mary Yates</em> all share DNA with you, the relationship is biologically confirmed.</p><div style="text-align:center;margin-top:20px;"><a href="dna_network.shtml" style="display:inline-block;padding:10px 20px;background:#f9a825;color:#333;text-decoration:none;border-radius:4px;font-weight:bold;">View the Network</a></div></div></div><div style="background:#e0f2f1;padding:30px;border-radius:8px;border:1px solid #b2dfdb;margin-bottom:40px;"><h2 style="color:#004d40;margin-top:0;text-align:center;">From Theory to Tools</h2><p style="text-align:center;max-width:700px;margin:0 auto 20px auto;">We have built a suite of forensic tools to visualize this data. Instead of raw spreadsheets, we offer interactive dashboards to prove your connection.</p><div style="display:flex;flex-wrap:wrap;justify-content:center;gap:15px;margin-top:20px;"><a href="ons_yates_dna_register.shtml" style="background:#006064;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">The Register</a><a href="lineage_proof.html" style="background:#00838f;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">Proof Engine</a><a href="dna_dossier.html" style="background:#00acc1;color:white;padding:12px 20px;text-decoration:none;border-radius:4px;font-weight:bold;">Forensic Dossier</a></div></div><div style="background:#f5f5f5;padding:20px;border-radius:8px;border:1px solid #ddd;"><h3 style="color:#555;margin-top:0;">Legacy Data: Y-DNA Haplogroups</h3><p style="font-size:0.9em;color:#666;margin-bottom:15px;">Y-DNA is the backbone of deep ancestry (27,000 BCE to 1600 AD). While not our primary focus for recent genealogy, we maintain a detailed record of the Yates Y-Chromosome mutations (R-M207 through FT266579).</p><a href="https://yates.one-name.net/gengen/dna_proof_y.htm" style="color:#006064;font-weight:bold;text-decoration:none;">&raquo; View Detailed Y-DNA Findings</a></div></div>"""
    pages_to_upload["dna_theory_of_the_case.htm"] = make_page("The Yates DNA Strategy", THEORY_CONTENT, False, stats_bar_full)

    GLOSSARY_CONTENT = r"""<div style="max-width:900px;margin:30px auto;font-family:'Segoe UI',sans-serif;">
    <p style="text-align:center;font-size:1.1em;color:#555;margin-bottom:30px;">Click on any term below to view its definition.</p>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Tester-Participant-MASKED (The Trigger)</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">The unique privacy code extracted from the user's NPFX tag.</div></details>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Tester-Participant-Unmasked</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">The real name of the tester.</div></details>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Platinum Standard</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">Lineages with 30+ matches and 10+ unique sources. Biologically proven.</div></details>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Keystone Tester</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">A high-value participant (15+ matches) who anchors a specific branch.</div></details>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Forensic Handshake</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">An informal term in genetic genealogy describing a confirmation pattern in which multiple independent DNA matches support the same ancestral line or family connection, strengthening confidence in an identification or relationship conclusion. Rather than relying on a single match, genealogists look for several matches that converge on the same family network, creating a “handshake” of agreement between genetic evidence and documentary research. This concept is especially valuable in unknown-parentage and forensic cases, where conclusions must be supported by redundant evidence.<br><br>A forensic handshake is often achieved through related methods such as triangulation, where three or more individuals share the same DNA segment from a common ancestor, and cluster analysis, which groups matches who also match one another and often represent a shared ancestral couple or lineage. Together, these approaches help meet an emerging genetic genealogy proof standard, emphasizing that reliable conclusions require multiple corroborating matches, consistency with records, and the exclusion of alternate explanations.</div></details>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Spanish Naming System</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">A traditional Hispanic naming convention in which an individual bears one or more given names followed by two surnames: the first inherited from the father (paternal surname) and the second from the mother (maternal surname). This system is historically rooted in Spain and is especially useful in genealogy because it preserves both parental lineages and improves identification in historical records.</div></details>
    <details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;font-size:1.1em;">Née</summary><div style="padding:15px;background:#fafafa;border-top:1px solid #ddd;color:#333;line-height:1.6;">A term meaning “born as,” used to indicate a woman’s maiden or birth surname before marriage. In genealogical and historical records, née identifies the surname a woman carried in her natal family line, preserving her connection to her parents and ancestry. For example, “Maria García, née López” shows that María’s birth surname was López, even though she later used García after marriage.<br><br>The use of née is especially important for tracing maternal family lines, since women’s surnames may change across generations in many cultures. By recording a woman’s birth name, genealogists can correctly link her to her original family, distinguish between individuals with similar married names, and maintain continuity in lineage reconstruction, particularly in marriage, probate, and church records.</div></details>
    </div>"""
    GLOSS_CSS = r"""<style>summary::-webkit-details-marker{display:none}summary{outline:none}</style>"""
    pages_to_upload["data_glossary.shtml"] = make_page("Data Glossary", GLOSSARY_CONTENT, False, stats_bar_full, extra_css=GLOSS_CSS)

    # ==============================================================================
    # 2. FORENSIC TOOL PAGES
    # ==============================================================================

    # 🌟 THE FIX: The JS algorithm now correctly scans the ENTIRE database (DB.forEach) to calculate independent corroborators!
    BIO_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Biological Proof Register</title><link rel="stylesheet" href="partials_unified.css"><style>__CSS_BASE__ .formal-brief { font-family: 'Georgia', serif; color: #000; } .formal-header { text-align: center; border-bottom: 3px solid #000; padding-bottom: 20px; margin-bottom: 30px; } .formal-header h1 { margin: 0; text-transform: uppercase; font-size: 26px; letter-spacing: 1px; } .formal-header p { margin: 5px 0 0 0; font-style: italic; font-size: 16px; color: #444; } .methodology-block { background: #f9f9f9; padding: 20px; border-left: 4px solid #455a64; margin-bottom: 30px; font-size: 15px; line-height: 1.6; } select { padding: 12px; font-size: 16px; width: 100%; max-width: 600px; margin-bottom: 20px; border: 1px solid #455a64; border-radius: 4px; } .control-panel { background: #eceff1; padding: 25px; border-radius: 8px; border: 1px solid #cfd8dc; margin-bottom: 20px; text-align: center; font-family: 'Segoe UI', sans-serif; } .tabs { display: flex; gap: 10px; margin-bottom: 15px; border-bottom: 2px solid #ddd; font-family: 'Segoe UI', sans-serif; } .tab { padding: 10px 20px; cursor: pointer; background: #eee; border-radius: 5px 5px 0 0; font-weight: bold; flex: 1; text-align: center; color: #555; } .tab.active { background: #455a64; color: white; } @media print { .methodology-block { border-left: 4px solid #000; background: transparent; } } .only-print { display: none; } @media print { .only-print { display: block !important; } }</style></head><body><div class="wrap"><h1 class="centerline no-print">📜 Biological Proof Register</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div class="no-print" style="float:right; margin-bottom:20px;"><button onclick="window.print()" style="background:#455a64;color:white;border:none;padding:10px 20px;border-radius:4px;cursor:pointer;font-weight:bold;font-size:14px;box-shadow:0 2px 4px rgba(0,0,0,0.2);">🖨️ Print Formal Brief</button></div><div class="no-print tabs"><div class="tab active" onclick="setMode('participant', event)">1. Generate Formal Brief (Tester)</div><div class="tab" onclick="setMode('ancestor', event)">2. View Ancestral Cohort</div></div><div class="no-print control-panel"><div id="panel-participant"><label style="font-weight:bold;color:#263238;display:block;margin-bottom:10px;font-size:1.1em;">Select Subject Tester for Official Report:</label><div style="text-align:center;"><select id="hpParticipantSelect" onchange="runApexPath()"><option value="">-- Choose Participant --</option></select></div></div><div id="panel-ancestor" style="display:none; text-align:center;"><label style="font-weight:bold;color:#263238;display:block;margin-bottom:5px;">Select Target Ancestor:</label><select id="hpAncestorSelect" onchange="runAncestorMap()"><option value="">-- Select Ancestor --</option></select></div></div><div id="proof-result" style="display:none;"></div></div></div><script>__JS_GLOBALS__ const partSel = document.getElementById('hpParticipantSelect'); const allTesters = Object.keys(DATA.participants); allTesters.sort((a,b) => DATA.participants[a].sort_key.localeCompare(DATA.participants[b].sort_key)); allTesters.forEach(t => { const o = document.createElement('option'); o.value = t; o.innerText = t; partSel.appendChild(o); }); const ancSel = document.getElementById('hpAncestorSelect'); Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[b].name.localeCompare(DATA.ancestors[a].name)).forEach(k => { const o = document.createElement('option'); o.value = k; o.innerText = DATA.ancestors[k].name; ancSel.appendChild(o); }); function setMode(m, e){ document.querySelectorAll('.tab').forEach(t => t.classList.remove('active')); if(e) e.target.classList.add('active'); document.getElementById('panel-ancestor').style.display = (m==='ancestor') ? 'block' : 'none'; document.getElementById('panel-participant').style.display = (m==='participant') ? 'block' : 'none'; document.getElementById('proof-result').style.display = 'none'; } function runAncestorMap(){ const key = ancSel.value; if(!key){ document.getElementById('proof-result').style.display='none'; return; } const d = DATA.ancestors[key]; const rows = DB.filter(r => r.ancestor === d.name); let html = `<div class="formal-brief"><div class="formal-header"><h1>Descendant Cohort Validation</h1><p>Subject: ${d.name}</p></div>`; html += `<div class="methodology-block"><strong>Status:</strong> The biological integrity of this ancestral couple has been assigned a status of <strong>${d.badge}</strong>. This is corroborated by <strong>${rows.length} independent matches</strong> sharing <strong>${d.cm} total cM</strong> of DNA.</div>`; html += `<table><thead><tr><th>Participant</th><th>Shared cM</th><th>Documented Lineage Path</th></tr></thead><tbody>`; rows.sort((a,b) => b.cm - a.cm).forEach(m => { html += `<tr><td><strong>${m.participant}</strong></td><td>${m.cm} cM</td><td style="font-size:0.9em;color:#444;">${m.lineage.split('->').join(' &rarr; ')}</td></tr>`; }); html += `</tbody></table></div>`; document.getElementById('proof-result').innerHTML = html; document.getElementById('proof-result').style.display = 'block'; } function runApexPath(){ const pName = partSel.value; if(!pName){ document.getElementById('proof-result').style.display='none'; return; } const myMatches = DB.filter(r => r.participant === pName); if(myMatches.length === 0) return alert("No match data found for this participant."); const testerPathNames = myMatches[0].t_names; const testerPathIDs = myMatches[0].t_ids; let finalHTML = `<div class="formal-brief"><div class="only-print" style="text-align:right; font-size:12px; color:#666; margin-bottom:20px;">__PRINT_STATS__</div><div class="formal-header"><h1>Forensic DNA Lineage Report</h1><p>Biological Verification via Collateral Saturation</p></div>`; if(testerPathNames && testerPathIDs) { const pNames = testerPathNames.split('|'); const pIDs = testerPathIDs.split('|'); const totalGens = pNames.length; const targetAncestor = pNames[0]; finalHTML += `<div class="methodology-block"><strong>SUBJECT TESTER:</strong> ${pName}<br><strong>TARGET ANCESTRAL NODE:</strong> ${targetAncestor}<br><br><strong>METHODOLOGY:</strong> This report validates the documented paper trail of the subject using <em>Collateral DNA Saturation</em>. The algorithm traces the subject's exact claimed lineage backwards from the present day. At each generational rung, the database is queried to identify independent biological cousins who intersect with the subject's DNA at that specific ancestral couple. A node is considered biologically proven when it is corroborated by redundant, independent lines of descent.</div>`; const summaryStr = pNames.join(' <br>&nbsp;&nbsp;&rdsh;&nbsp; '); finalHTML += `<h3 style="margin-top:30px; text-transform:uppercase; font-size:16px;">Claimed Documentary Lineage</h3><div style="padding:15px; border:1px solid #ddd; background:#fbfbfb; margin-bottom:40px; font-family:monospace; font-size:14px; color:#333;">${summaryStr}</div>`; let tableRowsHTML = ""; let highestHeat = 0; for(let i=0; i<totalGens; i++) { const idCombo = pIDs[i]; if(!idCombo) continue; const primaryID = idCombo.split('+')[0]; let nodeHeat = 0; DB.forEach(m => { if(m.search_ids && m.search_ids.split(',').includes(primaryID)) nodeHeat++; }); if(nodeHeat > highestHeat) highestHeat = nodeHeat; let badgeClass = "terminal"; let statusText = "Private Line"; if(nodeHeat >= 30) { badgeClass = "platinum"; statusText = "Confirmed Standard (30+)"; } else if(nodeHeat >= 15) { badgeClass = "gold"; statusText = "Confirmed Validation (15+)"; } else if(nodeHeat >= 5) { badgeClass = "silver"; statusText = "Verified Node (5+)"; } else if(nodeHeat >= 2) { badgeClass = "bronze"; statusText = "Emerging Node (2+)"; } if(i === totalGens - 1) { badgeClass = "descendant"; statusText = "Subject Tester"; } let displayID = idCombo.split('+').map(x => "I" + x.replace(/[^0-9]/g, '')).join(' & '); const genNum = totalGens - i; tableRowsHTML += `<tr><td style="font-weight:bold;text-align:center;">${genNum}</td><td><strong>${pNames[i]}</strong><br><span style="font-size:0.85em;color:#666;">ID: ${displayID}</span></td><td style="text-align:center; font-weight:bold; font-size:1.1em;">${nodeHeat}</td><td><span class="badge badge-${badgeClass}">${statusText}</span></td></tr>`; } let overallStatus = "Insufficient Data"; if(highestHeat >= 30) overallStatus = "Fully Validated (Platinum Standard)"; else if(highestHeat >= 15) overallStatus = "Strongly Validated (Gold Standard)"; else if(highestHeat >= 5) overallStatus = "Verified (Silver Standard)"; finalHTML += `<h3 style="margin-top:40px; text-transform:uppercase; font-size:16px;">Biological Corroboration Manifest</h3><p style="font-size:14px; color:#444;"><strong>Conclusion:</strong> The deep ancestry of this line is <strong>${overallStatus}</strong> based on the highest saturation point of ${highestHeat} independent collateral matches corroborating this lineage.</p><table><thead><tr><th style="text-align:center;width:60px;">Gen</th><th>Ancestor Node (Biological Couple)</th><th style="text-align:center;">Independent<br>DNA Corroborators</th><th>Forensic Status</th></tr></thead><tbody>${tableRowsHTML}</tbody></table></div>`; } else { finalHTML += `<p>Error: Could not locate documented spine for this tester in the GEDCOM.</p></div>`; } document.getElementById('proof-result').innerHTML = finalHTML; document.getElementById('proof-result').style.display = 'block'; }</script></body></html>"""
    pages_to_upload["biological_proof.html"] = BIO_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__PRINT_STATS__', stats_bar_full)

    PROOF_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Lineage Proof Engine</title><link rel="stylesheet" href="partials_unified.css"><style>body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}.proof-card{background:white;max-width:1000px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:30px}.badge-large{font-size:1.2em;padding:10px 20px;border-radius:30px;font-weight:bold;color:white;display:inline-block;margin-bottom:20px}.badge-platinum{background:#e5e4e2;color:#333;border:2px solid #ccc}.badge-gold{background:#ffd700;color:#b45f06}.badge-silver{background:#c0c0c0;color:#333}.badge-bronze{background:#cd7f32}.badge-descendant{background:#00bcd4;color:white;}.stats-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(150px,1fr));gap:15px;margin-bottom:30px}.stat-box{background:#f8f9fa;padding:15px;border-radius:8px;text-align:center;border:1px solid #ddd}.stat-val{font-size:1.8em;font-weight:bold;color:#006064}.stat-lbl{color:#666;font-size:0.9em;text-transform:uppercase}table{width:100%;border-collapse:collapse;margin-top:20px}th{background:#004d40;color:white;padding:12px;text-align:left}td{padding:10px;border-bottom:1px solid #eee}tr:hover{background-color:#f1f8e9}select{padding:10px;font-size:16px;width:100%;max-width:600px;margin-bottom:20px;border:1px solid #006064;border-radius:4px} .search-box{display:flex;gap:10px;max-width:600px;margin-bottom:20px;} input{padding:10px;font-size:16px;border:1px solid #ccc;border-radius:4px;flex:1;} button{background:#00838f;color:white;border:none;padding:0 20px;border-radius:4px;cursor:pointer;font-weight:bold;} button:hover{background:#006064} .clear-btn{background:#d32f2f;} .clear-btn:hover{background:#b71c1c;} .tabs{display:flex;gap:10px;margin-bottom:15px;border-bottom:2px solid #ddd;}.tab{padding:10px 20px;cursor:pointer;background:#eee;border-radius:5px 5px 0 0;font-weight:bold;flex:1;text-align:center;color:#555;}.tab.active{background:#00838f;color:white;} .control-panel{background:#e0f7fa;padding:20px;border-radius:8px;border:1px solid #b2ebf2;margin-bottom:20px} @media print { .no-print { display: none !important; } .only-print { display: block !important; } .proof-card { box-shadow: none; border: 1px solid #ccc; } body { background: white; } } .only-print { display: none; }</style></head><body><div class="wrap"><h1 class="centerline">🧬 Lineage Proof Engine</h1><div id="nav-slot">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div class="no-print" style="float:right;"><button onclick="window.print()" style="background:#455a64;color:white;border:none;padding:8px 15px;border-radius:4px;cursor:pointer;font-weight:bold;">🖨️ Print Report</button></div><div class="only-print" style="margin-bottom:20px; text-align:center; border-bottom:2px solid #004d40; padding-bottom:10px; font-weight:bold; color:#555;">__PRINT_STATS__</div><h3 style="color:#006064;margin-top:0;">Verify an Ancestral Line</h3><div class="tabs no-print"><div class="tab active" onclick="setMode('ancestor')">1. Select First Ancestor</div><div class="tab" onclick="setMode('probe')">2. Deep Probe by ID</div></div><div class="control-panel no-print"><div id="panel-ancestor"><label style="font-weight:bold;color:#006064;display:block;margin-bottom:5px;">Select Ancestor (Z-A Order):</label><select id="ancestorSelect" onchange="runAncestorProof()"><option value="">-- Choose Ancestor to Prove --</option></select></div><div id="panel-probe" style="display:none;"><label style="font-weight:bold;color:#006064;display:block;margin-bottom:5px;">Deep Probe by GEDCOM ID</label><div style="margin-bottom:10px;font-size:0.9em;color:#666;">Enter the exact numeric ID (e.g., <strong>1234</strong> for Person I1234).</div><div class="search-box"><input type="text" id="dictInput" placeholder="Enter ID number..." style="max-width:200px;"><button onclick="runDictSearch()">Probe ID</button><button class="clear-btn" onclick="clearResults()">Clear</button></div></div></div><div id="proof-result" style="display:none; margin-top:20px;"><div style="text-align:center"><span id="p-badge" class="badge-large"></span></div><div style="text-align:center;margin-bottom:15px;color:#004d40;font-size:1.2em;font-weight:bold;" id="p-title"></div><div class="stats-grid"><div class="stat-box"><div class="stat-val" id="p-matches">0</div><div class="stat-lbl">Matches</div></div><div class="stat-box"><div class="stat-val" id="p-testers">0</div><div class="stat-lbl">Unique Testers</div></div><div class="stat-box"><div class="stat-val" id="p-cm">0</div><div class="stat-lbl">Total cM</div></div><div class="stat-box"><div class="stat-val" id="p-integrity">0%</div><div class="stat-lbl">Integrity</div></div></div><div style="background:#e0f2f1;padding:15px;border-left:5px solid #004d40;margin-bottom:20px;font-style:italic;" id="p-verdict"></div><h4 style="border-bottom:2px solid #ddd;padding-bottom:10px;">Evidence Manifest</h4><div style="max-height:500px;overflow-y:auto;"><table id="evidence-table"><thead><tr><th>Participant</th><th>cM</th><th>Lineage Path</th></tr></thead><tbody></tbody></table></div></div></div></div><script>__JS_GLOBALS__ function cleanNum(str){let res="";for(let i=0;i<str.length;i++)if(str[i]>='0'&&str[i]<='9')res+=str[i];return res;} function clearResults(){ document.getElementById('proof-result').style.display='none'; document.getElementById('dictInput').value = ''; document.getElementById('ancestorSelect').value = ''; } function setMode(m){document.querySelectorAll('.tab').forEach(t=>t.classList.remove('active'));event.target.classList.add('active');document.getElementById('panel-ancestor').style.display=(m==='ancestor')?'block':'none';document.getElementById('panel-probe').style.display=(m==='probe')?'block':'none';clearResults();} const ancestorMap = new Map(); DB.forEach(r => { if(r.ancestor && r.id) ancestorMap.set(r.ancestor, cleanNum(r.id)); }); const sortedAncestors = Array.from(ancestorMap.keys()).sort((a, b) => b.localeCompare(a)); const ancSel = document.getElementById('ancestorSelect'); sortedAncestors.forEach(name => { const o = document.createElement('option'); o.value = ancestorMap.get(name); o.innerText = name; ancSel.appendChild(o); }); function generateProofReport(searchID, badgeLabel) { if(!searchID) return alert("Please enter a numeric ID."); const subMatches = DB.filter(r => { if(!r.search_ids) return false; return r.search_ids.split(',').some(token => cleanNum(token) === searchID); }); if(subMatches.length === 0) return alert("No matches found for ID: I" + searchID); let foundName = "Ancestor I" + searchID; if(subMatches.length > 0) { const r = subMatches[0]; const names = r.search_names.split('|'); const ids = r.search_ids.split(','); for(let i=0; i<Math.min(names.length, ids.length); i++){ if(cleanNum(ids[i]) === searchID) { foundName = names[i] + " (I" + searchID + ")"; break; } } } const matchCount = subMatches.length; const cmTotal = subMatches.reduce((acc, curr) => acc + (curr.cm || 0), 0); const uniqueTesters = new Set(subMatches.map(r => r.participant)).size; let integ = Math.min(100, (matchCount * 2) + (uniqueTesters * 10)); renderStats(badgeLabel, "Focus: " + foundName, matchCount, uniqueTesters, cmTotal, integ, "<strong>" + badgeLabel + " Analysis:</strong> Showing all descendants of " + foundName + " currently in the study.", subMatches); document.getElementById('proof-result').style.display='block'; } function runDictSearch(){ const rawInput = document.getElementById('dictInput').value; if(!rawInput) return; const searchID = cleanNum(rawInput); generateProofReport(searchID, "Deep Probe"); } function runAncestorProof() { const searchID = document.getElementById('ancestorSelect').value; if (!searchID) { clearResults(); return; } generateProofReport(searchID, "Ancestor Proof"); } function renderStats(badgeTxt, titleTxt, matches, testers, cm, integrity, verdict, rows){ const b = document.getElementById('p-badge'); b.className = 'badge-large badge-' + badgeTxt.toLowerCase().split(' ')[0]; if(badgeTxt === 'Deep Probe') b.style.backgroundColor = '#673ab7'; else if(badgeTxt === 'Ancestor Proof') b.style.backgroundColor = '#00838f'; b.innerText = badgeTxt; document.getElementById('p-title').innerText = titleTxt; document.getElementById('p-matches').innerText = matches; document.getElementById('p-testers').innerText = testers; document.getElementById('p-cm').innerText = cm.toLocaleString(); document.getElementById('p-integrity').innerText = integrity+'%'; document.getElementById('p-verdict').innerHTML = verdict; rows.sort((a,b)=>b.cm-a.cm); const tbody=document.querySelector('#evidence-table tbody');tbody.innerHTML=''; rows.forEach(m=>{const tr=document.createElement('tr');tr.innerHTML=`<td>${m.participant}</td><td>${m.cm}</td><td style="font-size:0.9em;color:#555;">${m.lineage.split('->').join(' &rarr; ')}</td>`;tbody.appendChild(tr);}); }</script></body></html>"""
    pages_to_upload["lineage_proof.html"] = PROOF_TMPL.replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__PRINT_STATS__', stats_bar_full)

    DOSS_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates DNA Forensic Dossier</title><link rel="stylesheet" href="partials_unified.css"><style>body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}.dossier-card{background:white;max-width:900px;margin:20px auto;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);overflow:hidden;position:relative}.dossier-header{background:#006064;color:white;padding:25px;text-align:center}.dossier-body{padding:30px}.badge{display:inline-block;padding:8px 15px;border-radius:20px;font-weight:bold;color:white;font-size:0.9em;margin-bottom:15px;text-transform:uppercase}.badge-platinum, .badge-keystone{background:#e5e4e2;color:#333;border:2px solid #ccc}.badge-gold{background:#ffd700;color:#b45f06}.badge-silver, .badge-active{background:#c0c0c0;color:#333}.badge-bronze, .badge-new{background:#cd7f32}.metric-grid{display:grid;grid-template-columns:1fr 1fr;gap:20px;margin-bottom:30px}.metric-box{background:#f8f9fa;padding:20px;border-radius:8px;text-align:center;border:1px solid #e9ecef}.metric-val{font-size:2em;font-weight:bold;color:#004d40}.metric-lbl{color:#666;font-size:0.9em;text-transform:uppercase}.progress-container{margin:20px 0}.progress-bar{background:#e9ecef;height:10px;border-radius:5px;overflow:hidden}.progress-fill{height:100%;background:#00838f;width:0%;transition:width 1s ease}.verdict-box{background:#e0f2f1;border-left:5px solid #004d40;padding:20px;margin:20px 0;font-family:'Georgia',serif;font-style:italic;color:#004d40;line-height:1.5}.contributors{margin-top:20px;border-top:1px solid #eee;padding-top:20px}.contributor-item{display:flex;justify-content:space-between;padding:8px 0;border-bottom:1px solid #f1f1f1}select{padding:10px;font-size:16px;width:100%;max-width:500px;margin:20px auto;display:block}.switch{text-align:center;margin:20px}.switch label{margin:0 15px;font-weight:bold;cursor:pointer;color:#006064}.action-btn{padding:10px 20px;background:#00838f;color:white;border:none;border-radius:4px;cursor:pointer;font-size:16px;margin:0 5px}.action-btn:hover{background:#006064}#composite-container{background:white;max-width:900px;margin:20px auto;padding:20px;border-radius:8px;box-shadow:0 4px 20px rgba(0,0,0,0.1);display:none;}.comp-table{width:100%;border-collapse:collapse;}.comp-table th{background:#004d40;color:white;padding:12px;text-align:left;}.comp-table td{padding:10px;border-bottom:1px solid #eee;}.footer-total{margin-top:15px;text-align:right;font-weight:bold;color:#004d40;border-top:2px solid #eee;padding-top:10px;}</style></head><body><div class="wrap"><h1 class="centerline">Yates DNA Forensic Dossier</h1><div id="nav-slot">__STATS_BAR____NAV_HTML__</div><div class="switch"><label><input type="radio" name="mode" value="ancestor" checked onchange="switchMode()"> Search by Ancestor</label><label><input type="radio" name="mode" value="participant" onchange="switchMode()"> Search by Participant (Kit)</label></div><div style="text-align:center;margin:30px;"><select id="dossierSelect" multiple size="10" style="height:auto;"><option value="">-- Select (Hold Ctrl/Shift for multiple) --</option></select><div style="margin-top:15px;"><button class="action-btn" onclick="addReport()">Add Reports</button><button class="action-btn" style="background:#d32f2f;" onclick="clearReports()">Clear All</button></div></div><div id="composite-container"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">Comparison Dashboard</h2><table class="comp-table"><thead><tr><th>Name</th><th>Role/Badge</th><th>Matches</th><th>Total cM</th><th>Integrity</th></tr></thead><tbody id="comp-body"></tbody></table><div id="comp-footer" class="footer-total"></div></div><div id="report-stack"></div></div><script>__JS_GLOBALS__ let currentMode = 'ancestor';let compTotalMatches = 0; let compTotalCM = 0; let compCount = 0; function switchMode(){currentMode = document.querySelector('input[name="mode"]:checked').value;populateDropdown();clearReports();} function populateDropdown(){const sel = document.getElementById('dossierSelect');sel.innerHTML = '';const source = (currentMode === 'ancestor') ? DATA.ancestors : DATA.participants;const sortedKeys = Object.keys(source).sort((a,b) => { return (currentMode === 'ancestor') ? source[b].name.localeCompare(source[a].name) : source[a].sort_key.localeCompare(source[b].sort_key); }); sortedKeys.forEach(key => { const opt = document.createElement('option'); opt.value = key; opt.innerText = source[key].name; sel.appendChild(opt); });} function clearReports(){document.getElementById('report-stack').innerHTML = '';document.getElementById('comp-body').innerHTML = '';document.getElementById('composite-container').style.display = 'none'; compTotalMatches=0; compTotalCM=0; compCount=0;} function addReport(){ const sel = document.getElementById('dossierSelect'); Array.from(sel.selectedOptions).forEach(option => { const key = option.value; if(!key) return; const d = (currentMode === 'ancestor') ? DATA.ancestors[key] : DATA.participants[key]; document.getElementById('composite-container').style.display = 'block'; compTotalMatches += d.matches; compTotalCM += d.cm; compCount++; const tr = document.createElement('tr'); tr.innerHTML = `<td><b>${d.name}</b></td><td>${d.badge || '-'}</td><td>${d.matches}</td><td>${d.cm.toLocaleString()}</td><td>${d.integrity}%</td>`; document.getElementById('comp-body').appendChild(tr); document.getElementById('comp-footer').innerText = `Composite Evidence: ${compCount} Profiles, ${compTotalMatches.toLocaleString()} Matches, ${compTotalCM.toLocaleString()} cM`; const badgeClass = (d.badge || 'bronze').toLowerCase().split(' ')[0]; const lblTesters = (currentMode==='ancestor') ? 'Unique Testers' : 'Study Rank'; const listTitle = (currentMode === 'ancestor') ? 'Top Contributors' : 'Ancestral Connections'; let listHTML = ''; for (const [name, count] of Object.entries(d.list_data)) {listHTML += `<div class="contributor-item"><span>${name}</span><span>${count} matches</span></div>`;} const html = `<div class="dossier-card" style="display:block; animation: fadeIn 0.5s;"><div class="dossier-header"><h2 style="margin:0">${d.name}</h2><div style="margin-top:10px;opacity:0.9">${currentMode.toUpperCase()} REPORT</div></div><div class="dossier-body"><div style="text-align:center"><span class="badge badge-${badgeClass}">${d.badge || 'N/A'}</span></div><div class="verdict-box"><strong>Forensic Analysis:</strong><br>${d.verdict}</div><div class="metric-grid"><div class="metric-box"><div class="metric-val">${d.matches}</div><div class="metric-lbl">Matches</div></div><div class="metric-box"><div class="metric-val">${d.testers}</div><div class="metric-lbl">${lblTesters}</div></div><div class="metric-box"><div class="metric-val">${d.cm.toLocaleString()}</div><div class="metric-lbl">Total cM</div></div><div class="metric-box"><div class="metric-val">${d.integrity}%</div><div class="metric-lbl">Integrity Score</div></div></div><div class="progress-container"><div style="display:flex;justify-content:space-between;margin-bottom:5px;font-size:0.9em;font-weight:bold;"><span>Score</span><span>${d.integrity}/100</span></div><div class="progress-bar"><div class="progress-fill" style="width:${d.integrity}%"></div></div></div><div class="contributors"><h3>${listTitle}</h3><div>${listHTML}</div></div></div></div>`; document.getElementById('report-stack').insertAdjacentHTML('afterbegin', html); }); } populateDropdown();</script></body></html>"""
    pages_to_upload["dna_dossier.html"] = DOSS_TMPL.replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS)

    BUST_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Yates Brick Wall Buster</title><link rel="stylesheet" href="partials_unified.css"><style>body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px}.dashboard{max-width:900px;margin:20px auto;background:white;padding:30px;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1)}.control-panel{background:#fff3e0;padding:20px;border-radius:8px;border:1px solid #ffcc80;margin-bottom:20px}.tabs{display:flex;gap:10px;margin-bottom:15px;border-bottom:2px solid #ddd;}.tab{padding:10px 20px;cursor:pointer;background:#eee;border-radius:5px 5px 0 0;font-weight:bold;flex:1;text-align:center;}.tab.active{background:#f57f17;color:white;}select{width:100%;padding:12px;font-size:16px;border:1px solid #f57c00;border-radius:4px;margin-top:5px}button{background:#e65100;color:white;border:none;padding:12px 25px;font-weight:bold;border-radius:4px;cursor:pointer;margin-top:10px;font-size:16px}button:hover{background:#ef6c00}.results-box{display:none;margin-top:30px}.finding-box{background:#e8eaf6;border-left:5px solid #3f51b5;padding:15px;margin-bottom:15px}.conclusion-box{background:#e0f2f1;border-left:5px solid #009688;padding:15px;margin-bottom:15px}.speculation-box{background:#ffebee;border-left:5px solid #e91e63;padding:15px;margin-bottom:15px}.checkbox-list{height:250px;overflow-y:scroll;border:1px solid #f57c00;background:white;border-radius:4px;padding:10px;}.checkbox-item{display:block;margin-bottom:5px;padding:5px;border-bottom:1px solid #eee;}.checkbox-item:hover{background-color:#ffe0b2;}.checkbox-item input{margin-right:10px;transform:scale(1.2);}.tooltip{cursor:pointer;font-size:1.2em;margin-left:10px;}.tooltip:hover{opacity:0.7;}.info-bubble{display:none;background:#e1f5fe;border:1px solid #81d4fa;padding:15px;border-radius:8px;margin-top:10px;color:#01579b;font-size:0.95em;}table{width:100%;border-collapse:collapse;margin-top:20px}th{background:#004d40;color:white;padding:12px;text-align:left}td{padding:10px;border-bottom:1px solid #eee}tr:hover{background-color:#f1f8e9}</style></head><body><div class="wrap"><h1 class="centerline">🧱 Brick Wall Buster</h1><div id="nav-slot">__STATS_BAR____NAV_HTML__</div><div class="dashboard"><p style="text-align:center;color:#555;margin-top:-10px;margin-bottom:20px;font-size:1.1em;">Predictive Forensic Analysis for Stalled Lineages</p><div class="tabs"><div class="tab active" onclick="setMode('ancestor')">1. Define by Ancestor</div><div class="tab" onclick="setMode('custom')">2. Define by Participants</div></div><div class="control-panel"><div id="panel-ancestor"><label style="font-weight:bold;color:#bf360c;">Select Your "End of Line" Ancestor:</label><select id="wallSelect"><option value="">-- Choose the ancestor you are stuck on --</option></select></div><div id="panel-custom" style="display:none;"><label style="font-weight:bold;color:#bf360c;">Select Testers (Scroll & Check): <span class="tooltip" onclick="toggleInfo()" title="Click to learn about Forensic Handshake Strategy">❓</span></label><div id="info-bubble" class="info-bubble"><strong>The Forensic Handshake Strategy:</strong><br>1. <strong>Isolate:</strong> We identify your specific group (the "Cluster").<br>2. <strong>Corroborate:</strong> We look for independent matches that converge on the same ancestral line.<br>3. <strong>Verify:</strong> A "Handshake" occurs when multiple testers confirm the same relationship, ruling out coincidence.</div><div id="testerList" class="checkbox-list"></div></div><div style="margin-top:15px;"><button onclick="runAnalysis()">🔨 Bust This Wall</button><button onclick="clearBuster()" style="background:#d32f2f;margin-left:10px;">Clear Selection & Results</button></div></div><div id="results" class="results-box"><div class="no-print" style="float:right;"><button onclick="window.print()" style="background:#455a64;font-size:14px;">🖨️ Print Official Report</button></div><h2 style="border-bottom:2px solid #ddd;padding-bottom:10px;margin-top:0;">Forensic Report</h2><div class="finding-box"><strong>🔍 Findings:</strong> <span id="txt-finding"></span></div><div class="conclusion-box"><strong>💡 Study Context:</strong> <span id="txt-conclusion"></span></div><div class="speculation-box"><strong>🚀 Forensic Handshake Prediction:</strong> <span id="txt-speculation"></span></div><div id="bridge-alert" style="display:none;background:#c8e6c9;border:1px solid #4caf50;padding:15px;margin-top:15px;color:#2e7d32;"></div><h3 style="color:#006064;border-bottom:2px solid #ddd;margin-top:30px;">Cluster Register</h3><div id="cluster-table-div"></div></div></div></div><script>__JS_GLOBALS__ let MODE='ancestor'; function toggleInfo(){const b=document.getElementById('info-bubble');b.style.display=(b.style.display==='block')?'none':'block';} const ancSel=document.getElementById('wallSelect'); Object.keys(DATA.ancestors).sort((a,b)=>DATA.ancestors[b].name.localeCompare(DATA.ancestors[a].name)).forEach(k=>{const o=document.createElement('option');o.value=k;o.innerText=DATA.ancestors[k].name;ancSel.appendChild(o);}); const testListDiv=document.getElementById('testerList'); const allTesters=Object.keys(DATA.participants); allTesters.sort((a,b)=> DATA.participants[a].sort_key.localeCompare(DATA.participants[b].sort_key)); let listHTML=""; allTesters.forEach(t=>{ listHTML+='<label class="checkbox-item"><input type="checkbox" value="'+t+'"> '+t+'</label>'; }); testListDiv.innerHTML=listHTML; function setMode(m){MODE=m;document.querySelectorAll('.tab').forEach(t=>t.classList.remove('active'));event.target.classList.add('active');document.getElementById('panel-ancestor').style.display=(m==='ancestor')?'block':'none';document.getElementById('panel-custom').style.display=(m==='custom')?'block':'none';document.getElementById('results').style.display='none';} function clearBuster(){ document.getElementById('wallSelect').selectedIndex=0; const cbs = document.querySelectorAll('#testerList input[type="checkbox"]'); cbs.forEach(cb => cb.checked = false); document.getElementById('results').style.display='none'; } function runAnalysis(){ let targetName="",clusterCount=0,clusterMembers=[],clusterMatches=0; if(MODE==='custom'){const checkedBoxes=document.querySelectorAll('#testerList input:checked');if(checkedBoxes.length===0)return alert("Select at least one tester.");clusterMembers=Array.from(checkedBoxes).map(cb=>cb.value);targetName="Custom Group ("+clusterMembers.length+" Testers)";} else {const key=ancSel.value;if(!key)return;const d=DATA.ancestors[key];targetName=d.name;clusterMembers=Object.keys(d.list_data);} clusterCount=clusterMembers.length;const groupRows=DB.filter(r=>clusterMembers.includes(r.participant));clusterMatches=groupRows.length;groupRows.sort((a,b)=> b.cm - a.cm);let tbl = '<table class="dataframe"><thead><tr><th>Participant</th><th>cM</th><th>Lineage</th></tr></thead><tbody>';groupRows.forEach(r => {tbl += `<tr><td>${r.participant}</td><td>${r.cm}</td><td style="font-size:0.9em;color:#555;">${r.lineage.split('->').join(' &rarr; ')}</td></tr>`;});tbl += '</tbody></table>';document.getElementById('cluster-table-div').innerHTML = tbl;const allMatches=DB.length,outsideMatches=allMatches-clusterMatches,pct=((clusterMatches/allMatches)*100).toFixed(2);let bridgeHTML="";const clusterRows=DB.filter(r=>clusterMembers.includes(r.participant));const bridges=clusterRows.filter(r=>r.ancestor!==targetName&&r.ancestor!==null);if(bridges.length>0){const uniqueBridges=[...new Set(bridges.map(b=>b.participant+" -> "+b.ancestor))];if(uniqueBridges.length>0){bridgeHTML=`<strong>🌉 BRIDGE DETECTED (Forensic Handshake Verified):</strong><br>A "Bridge" is a participant in your cluster who matches YOU but is <em>also</em> confirmed to belong to a known lineage in the database. We found <strong>${uniqueBridges.length}</strong> specific bridge connections.<br><br>These testers act as the "Forensic Handshake" linking your group to the following families:<br><ul>`;uniqueBridges.slice(0,5).forEach(b=>bridgeHTML+=`<li>${b}</li>`);bridgeHTML+=`</ul>`;document.getElementById('bridge-alert').innerHTML=bridgeHTML;document.getElementById('bridge-alert').style.display='block';}else{document.getElementById('bridge-alert').style.display='none';}}else{document.getElementById('bridge-alert').style.display='none';}const memStr=(clusterMembers.length>5)?clusterMembers.slice(0,5).join(", ")+"...":clusterMembers.join(", ");let findHTML=`<ul style="margin-top:0;padding-left:20px;"><li><strong>Target Group:</strong> ${targetName}</li><li><strong>Participants:</strong> ${clusterCount} (${memStr})</li><li><strong>Data Volume:</strong> ${clusterMatches} confirmed matches within this group.</li></ul>`;document.getElementById('txt-finding').innerHTML=findHTML;document.getElementById('txt-conclusion').innerHTML=`This cluster represents <strong>${pct}%</strong> of the total Yates database. There are <strong>${outsideMatches.toLocaleString()} other matches</strong> in the study.`;let otherGroups=[];for(const[k,v]of Object.entries(DATA.ancestors)){if(v.name!==targetName&&v.matches>=5)otherGroups.push(v);}otherGroups.sort((a,b)=>b.matches-a.matches);const top3=otherGroups.slice(0,3);let html="";if(top3.length===0){html+="<p>No strong signals found. The database currently lacks a 'Proven Line' (5+ matches) that connects to your group.</p>";}else{html+=`<p><strong>Methodology: The Forensic Handshake.</strong> We analyzed your cluster for "Handshakes"—patterns where multiple independent matches converge on the same ancestor to confirm a biological link. These are your most probable lineages:</p><ul>`;top3.forEach(g=>{html+=`<li><strong>${g.name}</strong>: Validated by <strong>${g.matches} Handshake Connections</strong> (shared match instances) within your cluster. (Probability: <span style="color:green">High</span>)</li>`;});html+=`</ul>`;}document.getElementById('txt-speculation').innerHTML=html;document.getElementById('results').style.display='block';}</script></body></html>"""
    pages_to_upload["brick_wall_buster.shtml"] = BUST_TMPL.replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS)

    # ==============================================================================
    # 3. REGISTERS (MAIN AND SINGLETONS)
    # ==============================================================================
    REGISTER_CSS = r"""<style>.wrap { max-width: 1400px !important; width: 95% !important; margin: 0 auto; } .table-scroll-wrapper { margin: 0 auto !important; width: 100% !important; max-width: 1200px !important; display: block; } table.dataframe { margin: 0 auto !important; width: 100% !important; text-align: left; border-collapse: collapse; } th, td { padding: 10px; text-align: left; } th { background: #004d40; color: white; }</style>"""
    TREE_CSS = r"""<style>.wrap { max-width: 1800px !important; width: 98% !important; margin: 0 auto; } .table-scroll-wrapper { margin: 0 auto !important; width: 100% !important; display: block; } table.dataframe { margin: 0 auto !important; width: 100% !important; text-align: left; border-collapse: collapse; font-size: 15px; } th, td { padding: 8px; text-align: left; line-height: 1.4; } th { background: #004d40; color: white; }</style>"""

    sb_str_anc = r"""<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div><div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print Register</button></div><div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;max-width:600px;margin-left:auto;margin-right:auto;border-radius:4px;"><strong>Sort Register:</strong> &nbsp;<span style="font-weight:bold;color:#006064;">By Ancestral Line</span> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1;text-decoration:none;">By Participant Name</a></div>"""
    sb_str_part = r"""<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div><div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print Register</button></div><div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;max-width:600px;margin-left:auto;margin-right:auto;border-radius:4px;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1;text-decoration:none;">By Ancestral Line</a> &nbsp;|&nbsp; <span style="font-weight:bold;color:#006064;">By Participant Name</span></div>"""

    sb_str_single_anc = r"""<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div><div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print Register</button></div><div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;max-width:600px;margin-left:auto;margin-right:auto;border-radius:4px;"><strong>Sort Singletons:</strong> &nbsp;<span style="font-weight:bold;color:#006064;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="admin_singletons_participants.shtml" style="color:#00acc1;text-decoration:none;">By Participant Name</a></div>"""
    sb_str_single_part = r"""<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div><div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print Register</button></div><div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;max-width:600px;margin-left:auto;margin-right:auto;border-radius:4px;"><strong>Sort Singletons:</strong> &nbsp;<a href="admin_singletons.shtml" style="color:#00acc1;text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="font-weight:bold;color:#006064;">By Participant Name</span></div>"""

    tree_za_toggle = r"""<div class="no-print" style="text-align:center;font-family:sans-serif;font-size:16px;margin:15px auto 20px auto;background:#e0f7fa;padding:10px;border:1px solid #b2ebf2;max-width:600px;border-radius:4px;"><strong>Individual Yates Family trees:</strong> &nbsp; <span style="font-weight:bold;color:#006064;">Z-A</span> &nbsp;|&nbsp; <a href="just-trees-az.shtml" style="color:#00acc1;text-decoration:underline;">A-Z</a></div>"""
    tree_az_toggle = r"""<div class="no-print" style="text-align:center;font-family:sans-serif;font-size:16px;margin:15px auto 20px auto;background:#e0f7fa;padding:10px;border:1px solid #b2ebf2;max-width:600px;border-radius:4px;"><strong>Individual Yates Family trees:</strong> &nbsp; <a href="just-trees.shtml" style="color:#00acc1;text-decoration:underline;">Z-A</a> &nbsp;|&nbsp; <span style="font-weight:bold;color:#006064;">A-Z</span></div>"""


    df_p = df.copy()
    df_p['sort_key'] = df_p['Unmasked'].apply(lambda n: re.split(r'\bnee\b|\bnée\b', str(n).split('[')[0].lower())[0].replace(',', '').replace('.', '').strip().split()[-1] if n else "zzz")
    df_p.sort_values(by=['sort_key', 'Match'], ascending=[True, True], inplace=True)
    df_p['Long_Narrative'] = df_p.apply(lambda r: f"<b>{format_last_first(r['Unmasked'])}</b> is a {r['cM']} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(r['ID'])}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{r['Match']}</b></a> via {r.get('Dir_Label', '').split('(')[0]} back {len(str(r.get('Lineage', '')).split('->'))} generations.", axis=1)
    df_p.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    pages_to_upload["ons_yates_dna_register_participants.shtml"] = make_page("ONS Yates Study DNA Register", sb_str_part + f'<div class="table-scroll-wrapper">{df_p.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)

    mc = df['Dir_Label'].value_counts()
    df_a = df[df['Dir_Label'].isin(mc[mc >= 2].index)].copy().sort_values(by=['Dir_Label', 'Lineage'], ascending=[True, True])
    df_a['Long_Narrative'] = df_a.apply(lambda r: f"{r['Unmasked']} is a {r['cM']} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(r['ID'])}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{r['Match']}</b></a> via {r.get('Dir_Label', '').split('(')[0]} back {len(str(r.get('Lineage', '')).split('->'))} generations.", axis=1)
    df_a.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    sbar_a = f"""<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Last updated:</strong> {timestamp} &nbsp;|&nbsp; <strong>Validated Matches (2+):</strong> {len(df_a):,} <span style="color:#d32f2f;">(Singleton matches hidden)</span></div>"""
    pages_to_upload["ons_yates_dna_register.shtml"] = make_page("ONS Yates Study DNA Register", sb_str_anc + f'<div class="table-scroll-wrapper">{df_a.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, sbar_a, extra_css=REGISTER_CSS)
    pages_to_upload["yates_ancestor_register.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]

    df_s = df[df['Dir_Label'].isin(mc[mc == 1].index)].copy()
    stats_bar_single = f"""<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>HOUSEKEEPING VIEW:</strong> Showing {len(df_s):,} singleton matches.</div>"""

    df_s_anc = df_s.sort_values(by=['Dir_Label'], ascending=False).copy()
    df_s_anc['Long_Narrative'] = df_s_anc.apply(lambda r: f"{r['Unmasked']} is a {r['cM']} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(r['ID'])}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{r['Match']}</b></a> via {r.get('Dir_Label', '').split('(')[0]} back {len(str(r.get('Lineage', '')).split('->'))} generations.", axis=1)
    df_s_anc.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    pages_to_upload["admin_singletons.shtml"] = make_page("Singleton Match Register", sb_str_single_anc + f'<div class="table-scroll-wrapper">{df_s_anc.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_single, extra_css=REGISTER_CSS)

    df_s_part = df_s.copy()
    df_s_part['sort_key'] = df_s_part['Unmasked'].apply(lambda n: re.split(r'\bnee\b|\bnée\b', str(n).split('[')[0].lower())[0].replace(',', '').replace('.', '').strip().split()[-1] if n else "zzz")
    df_s_part.sort_values(by=['sort_key', 'Match'], ascending=[True, True], inplace=True)
    df_s_part['Long_Narrative'] = df_s_part.apply(lambda r: f"<b>{format_last_first(r['Unmasked'])}</b> is a {r['cM']} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(r['ID'])}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{r['Match']}</b></a> via {r.get('Dir_Label', '').split('(')[0]} back {len(str(r.get('Lineage', '')).split('->'))} generations.", axis=1)
    df_s_part.rename(columns={'Long_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    pages_to_upload["admin_singletons_participants.shtml"] = make_page("Singleton Match Register", sb_str_single_part + f'<div class="table-scroll-wrapper">{df_s_part.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_single, extra_css=REGISTER_CSS)

    df_tree = df_a[['Linked_Tree_Line', 'Dir_Label']].copy()
    df_tree.rename(columns={'Linked_Tree_Line': 'TEMP'}, inplace=True)
    df_tree.sort_values(by=['Dir_Label'], ascending=[False], inplace=True)
    pages_to_upload["just-trees.shtml"] = make_page("Ancestor Register (Trees View)", tree_za_toggle + f'<div class="table-scroll-wrapper">{df_tree[["TEMP"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table").replace("<th>TEMP</th>", "<th>&nbsp;</th>")}</div>', True, sbar_a, extra_css=TREE_CSS)
    df_tree.sort_values(by=['Dir_Label'], ascending=[True], inplace=True)
    pages_to_upload["just-trees-az.shtml"] = make_page("Ancestor Register (Trees View)", tree_az_toggle + f'<div class="table-scroll-wrapper">{df_tree[["TEMP"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table").replace("<th>TEMP</th>", "<th>&nbsp;</th>")}</div>', True, sbar_a, extra_css=TREE_CSS)

    net_buf = []
    for anc, g in sorted(df.groupby('Dir_Label'), key=lambda x: len(x[1]), reverse=True):
        if len(g) < 2: continue
        net_buf.append(f"""<details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">{anc}</span> <span style="float:right;color:#004d40;font-size:0.9em;">Matches: {len(g)} | Total cM: {g['cM'].sum()}</span></summary><div style="padding:15px;"><div style="background:#fffde7;border-left:6px solid #fbc02d;padding:10px;margin-bottom:15px;font-family:sans-serif;color:#333;font-size:0.95em;"><strong>Collateral Saturation Analysis:</strong> Validated by <b>{len(g['Unmasked'].unique())} independent testers</b>.</div><table class="dataframe" border="1"><thead><tr style="text-align:left;"><th>Tester</th><th>cM</th><th>Lineage</th></tr></thead><tbody>""")
        for _, r in g.sort_values('cM', ascending=False).iterrows(): net_buf.append(f"<tr><td>{r['Unmasked']}</td><td>{r['cM']}</td><td>{r['Lineage']}</td></tr>")
        net_buf.append("</tbody></table></div></details>")

    NET_TMPL = f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Participating DNA Network</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css"><style>summary::-webkit-details-marker{{display:none}}summary{{outline:none}}</style></head><body id="top"><div class="wrap"><h1 class="centerline">Participating DNA Network</h1><div id="nav-slot">{stats_bar_full}{NAV_HTML}</div>{SITE_INFO}<div style="margin:20px auto;max-width:1400px;width:95%;">{"".join(net_buf)}</div></div>{JS_CORE}{BTT_BTN}</body></html>"""
    pages_to_upload["dna_network.shtml"] = NET_TMPL

    for fn, content in pages_to_upload.items():
        with open(fn, "w", encoding="utf-8") as f: f.write(content)
    print("    ✅ Core Registers and Static Pages Built Locally.")

    print("\n[PHASE 3] Uploading via FTP to Live Server...")
    try:
        ftps = FTP_TLS()
        ftps.connect(HOST, 21)
        ftps.auth()
        ftps.login(USER, PASS)
        ftps.prot_p()

        found_dir = False
        for d in [f"/{REMOTE_SUBDIR}", f"/public_html/{REMOTE_SUBDIR}", f"htdocs/{REMOTE_SUBDIR}", REMOTE_SUBDIR]:
            try:
                ftps.cwd(d)
                found_dir = True
                break
            except: pass

        if not found_dir:
            print("❌ FTP Directory Not Found.")
        else:
            files_to_upload = [f for f in os.listdir('.') if f.endswith(('.html', '.shtml', '.htm')) or f == CSV_DB]
            upload_count = 0
            for fn in files_to_upload:
                if os.path.exists(fn):
                    with open(fn, "rb") as fh:
                        ftps.storbinary(f"STOR {fn}", fh)
                    print(f"    📤 Uploaded: {fn}")
                    upload_count += 1
            print(f"\n🎉 MASTER PIPELINE COMPLETE. Successfully uploaded {upload_count} files to the live site.")
        ftps.quit()
    except Exception as e:
        print(f"❌ Upload Failed: {e}")

print("✅ Cell 5 Loaded.")

✅ Cell 5 Loaded.


In [26]:
# @title [CELL 6] MASTER ORCHESTRATOR (Run This Button)
import os, sys
print("="*60)
print("      MASTER ORCHESTRATOR")
print("      (Running Engine -> Publisher -> Upload)")
print("="*60)

if 'run_engine' not in globals() or 'run_publisher' not in globals():
    print("❌ ERROR: Modules not loaded! Please run the Engine and Publisher setup cells first.")
else:
    print("\n>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...")
    try:
        run_engine()
        print("✅ PHASE 1 COMPLETE.")

        print("\n>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...")
        run_publisher()
        print("✅ PHASE 2 COMPLETE.")

        print("\n" + "="*60)
        print("      🏆 MASTER PIPELINE SUCCESSFUL")
        print("="*60)
    except Exception as e:
        print(f"\n❌ CRITICAL FAILURE: {e}")

      MASTER ORCHESTRATOR
      (Running Engine -> Publisher -> Upload)

>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...
      [CELL 3] ENGINE STARTING (V113 - CLEAN)...

[STEP 1] Setup...
    👉 Source: yates_study_2025.ged

[STEP 4] Tracing Lineages...

[SUCCESS] Engine V113 Complete. Saved 1712 verified matches to engine_database.csv.
✅ PHASE 1 COMPLETE.

>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...
      [CELL 5] PUBLISHER STARTING (Upgraded V8)...
    ✅ Core Registers and Static Pages Built Locally.

[PHASE 3] Uploading via FTP to Live Server...
    📤 Uploaded: contents.shtml
    📤 Uploaded: subscribe.shtml
    📤 Uploaded: ons_yates_dna_register.shtml
    📤 Uploaded: research_admin.html
    📤 Uploaded: brick_wall_buster.shtml
    📤 Uploaded: ons_yates_dna_register_participants.shtml
    📤 Uploaded: dna_dossier.html
    📤 Uploaded: engine_database.csv
    📤 Uploaded: share_dna.shtml
    📤 Uploaded: lineage_proof.html
    📤 Uploaded: admin_singletons_participants.shtml
    📤 Uploaded: dna_

In [ ]:
# @title [CELL The Time Machine (Archiver + Dropbox Sync)]
import zipfile
import os
import pytz
import time
from datetime import datetime
from google.colab import files
from google.colab import userdata

# --- 1. INSTALL DROPBOX (IF MISSING) ---
try:
    import dropbox
    from dropbox.exceptions import AuthError
except ImportError:
    os.system('pip install dropbox')
    import dropbox
    from dropbox.exceptions import AuthError

def run_archiver():
    print("="*60)
    print("      [CELL 6] MANUAL ARCHIVER + DROPBOX SYNC")
    print("="*60)

    # --- 2. CREATE ZIP (SAFE MODE) ---
    # We explicitly exclude .zip to prevent "Zip Bombs"
    extensions = ('.csv', '.shtml', '.html', '.json', '.js', '.css')
    files_to_pack = [f for f in os.listdir('.') if f.lower().endswith(extensions) and "sample_data" not in f]

    if not files_to_pack:
        print("❌ No generated files found! Run the Publisher (Cell 4) first.")
        return

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%Y-%m-%d_%H%M")
    zip_name = f"Yates_Study_Backup_{timestamp}.zip"

    print(f"📦 Compressing {len(files_to_pack)} files into {zip_name}...")
    try:
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
            for file in files_to_pack:
                zf.write(file)
        print(f"    ✅ Archive Created: {zip_name} ({os.path.getsize(zip_name)/1024:.1f} KB)")
    except Exception as e:
        print(f"    ❌ Compression Failed: {e}")
        return

    # --- 3. FTP UPLOAD (BACKUPS FOLDER) ---
    print("\n[STEP 2] Uploading to Web Server (FTP)...")
    try:
        from ftplib import FTP_TLS
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()

        try:
            ftps.cwd("/ons-study/backups")
        except:
            try:
                ftps.mkd("/ons-study/backups")
                ftps.cwd("/ons-study/backups")
            except:
                pass

        with open(zip_name, "rb") as fh:
            ftps.storbinary(f"STOR {zip_name}", fh)
        print(f"    ✅ FTP Success: /ons-study/backups/{zip_name}")
        ftps.quit()
    except Exception as e:
        print(f"    ⚠️ FTP Upload skipped: {e}")

    # --- 4. DROPBOX SYNC (NEW) ---
    print("\n[STEP 3] Syncing to Dropbox...")
    try:
        # Initialize with Refresh Token (Long-term access)
        dbx = dropbox.Dropbox(
            app_key=userdata.get('DBX_APP_KEY'),
            app_secret=userdata.get('DBX_APP_SECRET'),
            oauth2_refresh_token=userdata.get('DBX_REFRESH_TOKEN')
        )

        # Upload the Zip
        target_path = f"/Backups/{zip_name}"
        with open(zip_name, "rb") as f:
            dbx.files_upload(f.read(), target_path, mode=dropbox.files.WriteMode.overwrite)

        print(f"    ✅ Dropbox Success: {target_path}")

    except Exception as e:
        print(f"    ❌ Dropbox Upload Failed: {e}")
        print("       (Check DBX_APP_KEY, DBX_APP_SECRET, DBX_REFRESH_TOKEN in Colab Secrets)")

    # --- 5. LOCAL DOWNLOAD (SAFETY NET) ---
    print("\n[STEP 4] Triggering Local Download...")
    try:
        files.download(zip_name)
    except Exception as e:
        print(f"    ⚠️ Auto-download blocked: {e}")

    print("✅ Archival Process Complete.")

# Run it
run_archiver()

      [CELL 6] MANUAL ARCHIVER + DROPBOX SYNC
📦 Compressing 17 files into Yates_Study_Backup_2026-02-20_0901.zip...
    ✅ Archive Created: Yates_Study_Backup_2026-02-20_0901.zip (737.3 KB)

[STEP 2] Uploading to Web Server (FTP)...
    ✅ FTP Success: /ons-study/backups/Yates_Study_Backup_2026-02-20_0901.zip

[STEP 3] Syncing to Dropbox...
    ✅ Dropbox Success: /Backups/Yates_Study_Backup_2026-02-20_0901.zip

[STEP 4] Triggering Local Download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Archival Process Complete.


In [ ]:
# @title [CELL Manual Zip & Download]
import os
import zipfile
import pytz
from datetime import datetime
from google.colab import files

print("="*60)
print("      [CELL 7] MANUAL ZIP & DOWNLOADER")
print("="*60)

# Create a timestamped zip file name
est = pytz.timezone('US/Eastern')
timestamp = datetime.now(est).strftime("%Y-%m-%d_%H%M")
zip_filename = f"Yates_Study_Manual_Upload_{timestamp}.zip"

# Find all the files we normally FTP
extensions = ('.html', '.shtml', '.htm', '.csv')
files_to_pack = [f for f in os.listdir('.') if f.lower().endswith(extensions) and "sample_data" not in f]

if not files_to_pack:
    print("❌ No files found to zip! Make sure you ran the Builder cells first.")
else:
    print(f"📦 Found {len(files_to_pack)} files. Compressing into {zip_filename}...\n")

    # Create the zip archive
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
        for file in files_to_pack:
            zf.write(file)
            print(f"  + Added: {file}")

    print(f"\n✅ Zip file created successfully! ({os.path.getsize(zip_filename)/1024:.1f} KB)")

    # Trigger the browser download
    print("⬇️ Prompting browser to download...")
    try:
        files.download(zip_filename)
        print("🎉 Download initiated! You can now manually upload these via FileZilla/Cyberduck.")
    except Exception as e:
        print(f"❌ Auto-download blocked by browser: {e}")
        print(f"👉 You can manually download '{zip_filename}' by clicking the Folder icon 📁 on the far left menu.")